<!-- # CNN autoencoder and Clustering from MTRX data

Use this notebook to load Scienta Omicron Matrix format SPM data and create standardised images for machine learning training and analysis. The code can generate both JPG image data, useful for manually checking the data, and windowed numpy data that can be loaded into ML models. 

The notebook then creates an autoencoder for training on a large dataset, followed by KMEANS clustering. 

**Author**: Steven R. Schofield  
**Created**: November, 2024 -->

# CASTEP - Si 001 surface - from code
## Steven R. Schofield (Universtiy College London) May 2025

### Determine appropriate paths whether we are working on macbook or the cluster

In [ ]:
import numpy as np 

# Supercell size
n_supercell = (1,1,3)

# Params
cut_off_energy = 700
spin_polarised = 'false'
nextra_bands = 12

# Cell params
kpoints_mp_grid = np.array([8, 4, 1])
vac = 15

# Job queue params
queue_name='A_192T_512G.q'
available_cores=192
available_memory='512G'
threads=8

a, b, c = n_supercell
s = f"{a}{b*2}{c}"

kx, ky, kz = kpoints_mp_grid
kp = f"{kx}x{ky}x{kz}"

job_folder = "si001_bulk"
filename = "si001_bulk_"+s
print(job_folder, "\n", filename)

si001_bulk 
 si001_bulk_123


In [170]:
import sys
from pathlib import Path

# Define candidate paths using Path objects
module_path_list = [
    Path('/Users/steven/academic-iCloud/Python/modules'),
    Path('/hpc/srs/Python/modules')
]

data_path_list = [
    Path('/Users/steven/Castep-data/work'),
    Path('/hpc/srs/castep')
]

# Resolve actual paths
module_path = next((p for p in module_path_list if p.exists()), None)
data_path = next((p for p in data_path_list if p.exists()), None)

# Check and report missing paths
if module_path is None:
    print("Error: Could not locate a valid module path.")
if data_path is None:
    print("Error: Could not locate a valid data path.")

if module_path is None or data_path is None:
    sys.exit(1)

# Add module_path to sys.path if needed
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))

# Print resolved paths
print(f"module_path = {module_path}")
print(f"data_path = {data_path}")

module_path = /Users/steven/academic-iCloud/Python/modules
data_path = /Users/steven/Castep-data/work


### Import modules

In [171]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

# Import custom module
import SRSCALCUTILS.castep_tools as ct

from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Paths for the calculation

In [172]:
# General
job_folder = job_folder
filename = filename

title = "Job folder: {}. Filename: {}.".format(job_folder, filename)
job_path = data_path / job_folder
verbose = False             # Set this True to print out more information

# Print resolved paths
print(f"job_path = {job_path}")

# If it doesn’t exist, create it (including any missing parent folders)
if not os.path.isdir(job_path):
    os.makedirs(job_path, exist_ok=True)

# Now safely switch into it
os.chdir(job_path)

#ct.delete_all_files_in_cwd(force=True)

job_path = /Users/steven/Castep-data/work/si001_bulk


### Base parameters of calculation and unit cell

In [173]:
params = {'task' : 'geomopt',
          'xc_functional'           : 'PBE',
          'cut_off_energy'          : cut_off_energy,
          'spin_polarised'          : spin_polarised,
          'write_cell_structure'    : 'true',
          'charge'                  : 0,
          'nextra_bands'            : nextra_bands,
          'geom_energy_tol'         : 0.00005,          # default: 0.00005 eV 
          'geom_disp_tol'           : 0.002,            # default: 0.002 Ang.
          'geom_max_iter'           : 100,              # default: 30
          'geom_convergence_win'    : 2,                # default: 2
          'max_scf_cycles'          : 300               # default: 30
          }              

cell_constraints = np.array([
            [1,  1,  2],
            [0,  0,  0]
        ])

kpoints_mp_grid = kpoints_mp_grid

fix_all_ions = True
symmetry_generate=True
symmetry_tol = 0.01

In [174]:
lattice_cart_bulk = np.array([
            [3.8641976,     0.0,     0.0],
            [0.0,     7.7283952,     0.0],
            [0.0,     0.0,     5.4648012]
        ])

positions_frac_bulk = np.array([   
   ['Si',       0.0000000000,    0.7500000000,    0.7500000000],
   ['Si',       0.0000000000,    0.2500000000,    0.7500000000],
   ['Si',       0.5000000000,    0.7500000000,    0.5000000000],
   ['Si',       0.5000000000,    0.2500000000,    0.5000000000],
   ['Si',       0.5000000000,    0.5000000000,    0.2500000000],
   ['Si',       0.5000000000,    0.0000000000,    0.2500000000],
   ['Si',       0.0000000000,    0.5000000000,    0.0000000000],
   ['Si',       0.0000000000,    0.0000000000,    0.0000000000]
], dtype=object)


### Create supercell -- programatically

In [175]:
n = n_supercell
positions_frac, lattice_cart = ct.create_supercell_from_fractional_coords(positions_frac_bulk,lattice_cart_bulk,n)
positions_frac = ct.sort_positions_frac(positions_frac)

### Add hydrogen termination to cell bottom 

In [176]:
# # First select the atoms on the bottom layer 
# criteria = ('min','min','min')
# selected_atom_frac, selected_atom_cart = ct.select_atom_by_conditions(positions_frac, lattice_cart, criteria)
# reference_position = selected_atom_cart[1:]
# plane = (0,0,1)
# labelled_positions_frac = ct.select_atoms_by_plane(
#     positions_frac,
#     lattice_cart,
#     plane,
#     reference_position,
#     tolerance=1
# )

# # Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
# phi = np.deg2rad(90)    
# si_h_bond = 1.5                         
# theta_h1 = np.deg2rad(180) - np.arccos(-1/3) / 2                
# theta_h2 = np.deg2rad(180) + np.arccos(-1/3) / 2            
# bond_h1  = ct.bond_vector_from_spherical(theta_h1, phi, si_h_bond)                   
# bond_h2  = ct.bond_vector_from_spherical(theta_h2, phi, si_h_bond)

# # Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
# positions_frac, lattice_cart = ct.add_atoms_to_positions_frac(
#                                     labelled_positions_frac,
#                                     lattice_cart,
#                                     [bond_h1,bond_h2],
#                                     extend_unit_cell=(0, 0, 1),
#                                     atom="H")
# positions_frac = ct.sort_positions_frac(positions_frac)

### Select the surface atoms

In [177]:
# # Find a surface atom to define the surface plane
# criteria = ('min','min','max')
# selected_atom_frac, selected_atom_cart = ct.select_atom_by_conditions(positions_frac, lattice_cart, criteria)
# reference_position = selected_atom_cart[1:]
# plane = (0,0,1)

# # Label the atoms at the 001 surface
# labelled_positions_frac_surf = ct.select_atoms_by_plane(
#     positions_frac,
#     lattice_cart,
#     plane,
#     reference_position,
#     tolerance=1
# )

# # Sublabel the two atoms of each dimer p(2x1)
# labelled_positions_frac_dim1, labelled_positions_frac_dim2 = ct.selected_toggle_plane_selection(
#     labelled_positions_frac_surf, 
#     fast='y', 
#     slow='x', 
#     alternate=False
# )

### Dimerise surface (2x1)

In [178]:
# # Dimerise surface
# disp_y = 0.734
# disp_z = 0.242
# v1 = (0,-disp_y,-disp_z)
# v2 = (0,disp_y,-disp_z)
# positions_frac = ct.selected_translate(labelled_positions_frac_dim1, lattice_cart, v1, return_labelled=False)

# labelled_positions_frac_dim2 = ct.update_labelled_positions_frac(labelled_positions_frac_dim2,positions_frac)
# positions_frac = ct.selected_translate(labelled_positions_frac_dim2, lattice_cart, v2, return_labelled=False)



### Dimerise surface (add c4x2) buckling)

In [179]:
# # Sublabel the two atoms of each dimer c(4x2)
# labelled_positions_frac_buc1, labelled_positions_frac_buc2 = ct.selected_toggle_plane_selection(
#     labelled_positions_frac_surf, 
#     fast='y', 
#     slow='x', 
#     alternate=True
# )

# disp_y = 0.0
# disp_z = 0.4
# v1 = (0,-disp_y,-disp_z)
# v2 = (0,disp_y,disp_z)

# labelled_positions_frac_buc1 = ct.update_labelled_positions_frac(labelled_positions_frac_buc1,positions_frac)
# positions_frac = ct.selected_translate(labelled_positions_frac_buc1, lattice_cart, v1, return_labelled=False)

# labelled_positions_frac_buc2 = ct.update_labelled_positions_frac(labelled_positions_frac_buc2,positions_frac)
# positions_frac = ct.selected_translate(labelled_positions_frac_buc2, lattice_cart, v2, return_labelled=False)


### Add hydrogen to the top

In [180]:
# # Updated surface labels with current surface positions after dimerisation
# labelled_positions_frac_surf = ct.update_labelled_positions_frac(labelled_positions_frac_surf,positions_frac)
# labelled_positions_frac_dim1 = ct.update_labelled_positions_frac(labelled_positions_frac_dim1,positions_frac)
# labelled_positions_frac_dim2 = ct.update_labelled_positions_frac(labelled_positions_frac_dim2,positions_frac)

# # Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
# phi = np.deg2rad(90)    
# si_h_bond = 1.5                         
# # theta_h1 = np.deg2rad(0) + np.arccos(-1/3) / 2                
# # theta_h2 = np.deg2rad(0) - np.arccos(-1/3) / 2            
# theta = 20 # degrees
# theta_h1 = np.deg2rad(theta)
# theta_h2 = np.deg2rad(360 - theta)
# bond_h1  = ct.bond_vector_from_spherical(theta_h1, phi, si_h_bond)                   
# bond_h2  = ct.bond_vector_from_spherical(theta_h2, phi, si_h_bond)

# # Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
# positions_frac_h1, lattice_cart = ct.add_atoms_to_positions_frac(
#                                     labelled_positions_frac_dim1,
#                                     lattice_cart,
#                                     bond_h1,
#                                     extend_unit_cell=(0, 0, 1),
#                                     atom="H")

# # Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
# positions_frac_h2, lattice_cart = ct.add_atoms_to_positions_frac(
#                                     labelled_positions_frac_dim2,
#                                     lattice_cart,
#                                     bond_h2,
#                                     extend_unit_cell=(0, 0, 1),
#                                     atom="H")

# # Merge the two sets of positions_frac for the two sets of surface hydrogen atoms and resort. 
# positions_frac = ct.merge_posfrac_or_labelled_posfrac(positions_frac_h1, positions_frac_h2)
# positions_frac = ct.sort_positions_frac(positions_frac)

### Add vacuum spacing

In [181]:
# vac = vac
# positions_frac, lattice_cart = ct.create_vacuum_spacing(positions_frac, lattice_cart, vac)

### Add ionic contraints

In [182]:
# # Create fixed ion constraints
# conditions = "z < 2.5"
# ionic_constraints = ct.select_atoms_by_region(positions_frac, lattice_cart, conditions)

In [183]:
castep_path = job_path / f"{filename}.castep"

print('=' * 80 + f'\nCASTEP file: {castep_path}\n' + '=' * 80 + '\n')   

param_filename = ct.write_param_file(
    params,
    title = title,
    filename=filename,
    path=job_path,
    display_file=True
    )

cell_filename = ct.write_cell_file(
    title = title,
    path=job_path,
    filename=filename,
    lattice_cart=lattice_cart,
    positions_frac=positions_frac,
    cell_constraints=cell_constraints,
 #   ionic_constraints=ionic_constraints,
    fix_all_ions=fix_all_ions,
    symmetry_generate=symmetry_generate,
    symmetry_tol = symmetry_tol,
    kpoints_mp_grid=kpoints_mp_grid,
    display_file=True
)


# job_filename= ct.write_job_script(
#     path=job_path,
#     filename=filename,
#     wall_time='72:00:00',
#     queue_name='A_192T_1024G.q',
#     available_cores=192,
#     available_memory='1024G',
#     threads=4,
#     display_file=True
# )

job_filename= ct.write_job_script(
    path=job_path,
    filename=filename,
    wall_time='72:00:00',
    queue_name=queue_name,
    available_cores=available_cores,
    available_memory=available_memory,
    threads=threads,
    display_file=True
)

# # Convert to cart and write xyz
# positions_cart = ct.frac_to_cart(lattice_cart, positions_frac)
# xyz = ct.write_xyz(positions_cart, path=job_path, filename=filename, comment='Converted by SRSCALCUTILS')


CASTEP file: /Users/steven/Castep-data/work/si001_bulk/si001_bulk_123.castep

Wrote param file to: /Users/steven/Castep-data/work/si001_bulk/si001_bulk_123.param
!TITLE: Job folder: si001_bulk. Filename: si001_bulk_123.

TASK                 : geomopt
XC_FUNCTIONAL        : PBE
CUT_OFF_ENERGY       : 700
SPIN_POLARISED       : false
WRITE_CELL_STRUCTURE : true
CHARGE               : 0
NEXTRA_BANDS         : 12
GEOM_ENERGY_TOL      : 5e-05
GEOM_DISP_TOL        : 0.002
GEOM_MAX_ITER        : 100
GEOM_CONVERGENCE_WIN : 2
MAX_SCF_CYCLES       : 300


Wrote cell file to: /Users/steven/Castep-data/work/si001_bulk/si001_bulk_123.cell
! Job folder: si001_bulk. Filename: si001_bulk_123.

%BLOCK lattice_cart
   ANG
       3.8641976000    0.0000000000    0.0000000000
       0.0000000000    7.7283952000    0.0000000000
       0.0000000000    0.0000000000   16.3944036000
%ENDBLOCK lattice_cart

%BLOCK CELL_CONSTRAINTS
       1    1    2
       0    0    0
%ENDBLOCK CELL_CONSTRAINTS

%BLOCK position

In [184]:
# if str(module_path) == '/hpc/srs/Python/modules':
#     !bash -l -c "mpirun -np 62 castep.mpi {filename}"
# else:
#     !zsh -l -c "castepmpi {filename}"
